# A) Numpy Implementation

In [1]:
import math
import numpy as np  
from download_mnist import load
import operator  
import time

In [2]:
x_train, y_train, x_test, y_test = load()

x_mean = np.mean(x_train, axis = 0)
x_std = np.std(x_train, axis = 0)

def standardize(data):
    stan_data = []
    for i in range(28*28):
        if x_std[i] == 0:
            stan_data.append(data[:,i] - x_mean[i])
        else:
            stan_data.append((data[:,i] - x_mean[i]) / x_std[i])
    return np.array(stan_data).T

x_train = standardize(x_train)
x_test = standardize(x_test)

def relu(Z):
    return np.maximum(0,Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis=1)[:,None]

In [16]:
def main():
    global x_train
    global y_train
    batch_size = 128
    FC1W =  0.01 * np.random.randn(28*28, 200)
    FC2W =  0.01 * np.random.randn(200, 50)
    FC3W = 0.01 * np.random.randn(50, 10)
    b1 = 0.0 * np.random.randn(1, 200)
    b2 = 0.0 * np.random.randn(1, 50)
    b3 = 0.0 * np.random.randn(1, 10)
    
    step_size = 0.01
    reg = 0.001
    num_examples = x_train.shape[0]
    
    for i in range(20):
        pem = np.random.permutation(x_train.shape[0])
        x_train = x_train[pem]
        y_train = y_train[pem]
        counter = 0
        while counter < x_train.shape[0]:
            x_train_batch = x_train[counter:counter + batch_size]
            y_train_batch = y_train[counter:counter + batch_size]
            num_examples = x_train_batch.shape[0]
            
            scores1 = np.dot(x_train_batch,FC1W) + b1
            relu1 = relu(scores1)
            scores2 = np.dot(relu1, FC2W) + b2
            relu2 = relu(scores2)
            scores3 = np.dot(relu2, FC3W) + b3

            probs = softmax(scores3)

            correct_logprobs = -np.log(probs[range(num_examples), y_train_batch])
            data_loss = np.sum(correct_logprobs)/num_examples
            reg_loss = 0.5*reg*(np.sum(FC1W*FC1W)+np.sum(FC2W*FC2W)+np.sum(FC3W*FC3W))
            loss = data_loss + reg_loss
            if counter == 0:
                print("Epoch %d: loss %f" % (i+1,loss))
            counter+=batch_size

            preds = np.argmax(probs, axis = 1)

            dscores3 = probs
            dscores3[range(num_examples),y_train_batch] -=1
            dscores3 /= num_examples


            dFC3W = np.dot(relu2.T, dscores3) + reg * FC3W
            db3 = np.sum(dscores3, axis = 0, keepdims=True)

            dRELU2 = np.dot(dscores3, FC3W.T)

            dRELU2[scores2 <= 0] = 0

            dFC2W = np.dot(relu1.T, dRELU2)+ reg * FC2W
            db2 = np.sum(dRELU2, axis = 0, keepdims=True)

            dRELU1 = np.dot(dRELU2, FC2W.T)

            dRELU1[scores1 <= 0] = 0

            dFC1W = np.dot(x_train_batch.T, dRELU1)+ reg * FC1W
            db1 = np.sum(dRELU1, axis = 0, keepdims=True)

            FC3W -= step_size * dFC3W
            b3 -= step_size * db3
            FC2W -= step_size * dFC2W
            b2 -= step_size * db2
            FC1W -= step_size * dFC1W
            b1 -= step_size * db1
        
        tscores1 = np.dot(x_test,FC1W) + b1
        trelu1 = relu(tscores1)
        tscores2 = np.dot(trelu1, FC2W) + b2
        trelu2 = relu(tscores2)
        tscores3 = np.dot(trelu2, FC3W) + b3

        tprobs = softmax(tscores3)
        tpreds = np.argmax(tprobs, axis = 1)
        num_correct = len(np.where(tpreds - y_test == 0)[0])
        print('Epoch %d test accuracy: %f' % ( i+1, num_correct/x_test.shape[0] * 100))
        
        
if __name__ == '__main__':
    start_time = time.time()
    main()
    print ("---execution time: %s seconds ---" % (time.time() - start_time))

epoch 1: loss 2.310808
Epoch 1 test accuracy: 11.370000
epoch 2: loss 2.308659
Epoch 2 test accuracy: 15.440000
epoch 3: loss 2.298739
Epoch 3 test accuracy: 29.690000
epoch 4: loss 2.134334
Epoch 4 test accuracy: 43.030000
epoch 5: loss 1.641118
Epoch 5 test accuracy: 68.360000
epoch 6: loss 1.021657
Epoch 6 test accuracy: 82.320000
epoch 7: loss 0.651701
Epoch 7 test accuracy: 87.430000
epoch 8: loss 0.392956
Epoch 8 test accuracy: 89.910000
epoch 9: loss 0.272943
Epoch 9 test accuracy: 90.900000
epoch 10: loss 0.344619
Epoch 10 test accuracy: 91.700000
epoch 11: loss 0.167442
Epoch 11 test accuracy: 92.330000
epoch 12: loss 0.219283
Epoch 12 test accuracy: 92.970000
epoch 13: loss 0.168937
Epoch 13 test accuracy: 93.280000
epoch 14: loss 0.138621
Epoch 14 test accuracy: 93.790000
epoch 15: loss 0.194145
Epoch 15 test accuracy: 94.030000
epoch 16: loss 0.215424
Epoch 16 test accuracy: 94.460000
epoch 17: loss 0.185918
Epoch 17 test accuracy: 94.680000
epoch 18: loss 0.172182
Epoch 18

# B) PyTorch Implementation

In [4]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [19]:
 def main():

    torch.manual_seed(90)

    device = torch.device("cpu")

    kwargs = {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=128, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=10000, shuffle=True, **kwargs)

    
    
    model = torch.nn.Sequential(
                torch.nn.Linear(28*28, 200),
                torch.nn.ReLU(),
                torch.nn.Linear(200, 50),
                torch.nn.ReLU(),
                torch.nn.Linear(50,10),
                torch.nn.Softmax(dim = 1)).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.001)
    for epoch in range(10):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.view(-1,28*28)
            optimizer.zero_grad()
            output = model(data)
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx == 0:
                print('Train Epoch: {}\tLoss: {:.6f}'.format(
                    epoch+1, loss.item()))
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                data = data.view(-1,28*28)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        print('\nTest set: Accuracy: {}/{} \n'.format(
            correct, len(test_loader.dataset)))
    
    
if __name__ == '__main__':
    start_time = time.time()
    main()
    print ("---execution time: %s seconds ---" % (time.time() - start_time))


Train Epoch: 1	Loss: 2.302934

Test set: Accuracy: 8353/10000 (84%)

Train Epoch: 2	Loss: 1.606482

Test set: Accuracy: 9142/10000 (91%)

Train Epoch: 3	Loss: 1.572431

Test set: Accuracy: 9267/10000 (93%)

Train Epoch: 4	Loss: 1.543095

Test set: Accuracy: 9337/10000 (93%)

Train Epoch: 5	Loss: 1.510418

Test set: Accuracy: 9389/10000 (94%)

Train Epoch: 6	Loss: 1.547799

Test set: Accuracy: 9472/10000 (95%)

Train Epoch: 7	Loss: 1.527524

Test set: Accuracy: 9497/10000 (95%)

Train Epoch: 8	Loss: 1.498596

Test set: Accuracy: 9508/10000 (95%)

Train Epoch: 9	Loss: 1.495493


KeyboardInterrupt: 